In [1]:
from hydra import compose, initialize

with initialize(version_base=None, config_path="./src/configs/datasets"):
    cfg = compose(config_name="sasrec")

cfg

{'train': {'_target_': 'src.datasets.YambdaDataset', 'name': 'train', 'inter_type': 'listens', 'yambda_size': '50m', 'min_inter_user': 1000, 'min_inter_item': 10, 'min_len': 5, 'q': 0.95, 'max_len': 200, 'instance_transforms': '${transforms.instance_transforms.inference}'}, 'test': {'_target_': 'src.datasets.YambdaDataset', 'name': 'test', 'inter_type': '${..train.inter_type}', 'yambda_size': '${..train.yambda_size}', 'min_inter_user': '${..train.min_inter_user}', 'min_inter_item': '${..train.min_inter_item}', 'min_len': '${..train.min_len}', 'q': '${..train.q}', 'max_len': '${..train.max_len}', 'instance_transforms': '${transforms.instance_transforms.inference}'}}

In [2]:
val_config = cfg["train"]
val_config["name"] = "test"
val_config["instance_transforms"] = None
val_config = dict(val_config)
val_config.pop("_target_")

'src.datasets.YambdaDataset'

In [3]:
from src.datasets import YambdaDataset

dataset = YambdaDataset(**val_config)

In [4]:
import torch

cnt = 0
k = 10
seen_items = set()

for data in dataset:
    values, counts = torch.unique(data["seq"], return_counts=True)
    top_k_indices = torch.argsort(counts, descending=True)[:k]
    recs = values[top_k_indices]
    seen_items |= set(recs.cpu().numpy())
    if torch.any(recs == data["item"]):
        cnt += 1

print(f"HR@{k}: {cnt / len(dataset)}, COV@{k}: {len(seen_items) / dataset.n_items}")

HR@10: 0.07359710460662708, COV@10: 0.45716593659192434
